In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import os
import sys
import tflearn
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

print(tf.VERSION)
print(tf.keras.__version__)

np.set_printoptions(suppress=True)

scaler = MinMaxScaler()

with open(
        '/Users/bakunowski/Documents/goldsmiths/2018_19/dissertation/project/neural/newest_data_1000_examples/MFCC.json'
) as f:
    d2 = json.load(f)

mfcc = np.array(d2["lowlevel"]["mfcc"])
scaler.fit(mfcc)
mfcc_norm = scaler.transform(mfcc)
# normalized between 0 and 1 and reshaped to 45 frames x 13 values
mfcc_norm_reshaped = mfcc_norm.reshape(1000, 45, 13)

with open('/Users/bakunowski/Documents/goldsmiths/2018_19/dissertation/project/neural/newest_data_1000_examples/parameters.json') as f:
    d1 = json.load(f)

duration = np.array(d1["parameters"]["duration"]).reshape(1000, 1)
numberOfGrains = np.array(d1["parameters"]["numberOfGrains"]).reshape(1000, 1)
pitch = np.array(d1["parameters"]["pitch"]).reshape(1000, 1)
position = np.array(d1["parameters"]["position"]).reshape(1000, 1)
spread = np.array(d1["parameters"]["spread"]).reshape(1000, 1)

parameters = np.concatenate((duration, numberOfGrains, pitch, position, spread), axis=1)

scaler.fit(parameters)
# normalized between 0 and 1
parameters_norm = scaler.transform(parameters)

X_train = mfcc_norm_reshaped[:800]
y_train = parameters_norm[:800]

X_test = mfcc_norm_reshaped[800:1000]
y_test = parameters_norm[800:1000]

np.set_printoptions(suppress=True)
print("Train dimensions", X_train.shape)
print("Train labels dimensions", y_train.shape)
print("Test dimensions", X_test.shape)
print("Test labels dimensions", y_test.shape)

model = tf.keras.Sequential([
    layers.Flatten(input_shape=(45,13)),
    layers.Dense(585, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(5)])

optimizer = tf.train.GradientDescentOptimizer(0.1)

model.compile(loss='mean_squared_error',
              optimizer=optimizer,
              metrics=['mean_squared_error', 'accuracy'])

model.summary()

example_batch = X_train[:10]
example_result = model.predict(example_batch)
print(example_result)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
1.13.1
2.2.4-tf
Train dimensions (800, 45, 13)
Train labels dimensions (800, 5)
Test dimensions (200, 45, 13)
Test labels dimensions (200, 5)
Instructions for updating:
Use tf.cast instead.


/Users/bakunowski/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 585)               0         
_________________________________________________________________
dense (Dense)                (None, 585)               342810    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               75008     
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 418,463
Trainable params: 418,463
Non-trainable params: 0
_________________________________________________________________
[[ 0.8230839   0.01789357 -0.07594353 -0.44434726  0.8169522 ]
 [ 0.6294199   0.17124881  0.0303061  -0.00426348  0.4145323 ]
 [ 0.6439167   0.39445645  0.05976285 -0.08385779  0.40885895]
 [ 0.64799345  0.27393317 -0.13996139 -0.27097583  0.4045952 ]
 [ 0.5158637   0

In [2]:
print(X_train[0].shape[0])
print(X_train[0].shape[1])
print(y_train[0].shape[0])

45
13
5


In [4]:
learning_rate = 0.001
training_iters = 40
batch_size = 64
number_hidden = 256
parameter_size = y_train[0].shape[0]
features_cols = X_train[0].shape[0]
features_rows = X_train[0].shape[1]

net = tflearn.input_data([None, 13, 45])
net = tflearn.lstm(net, number_hidden, dropout=0.8)
net = tflearn.fully_connected(net, parameter_size, activation='relu')
net = tflearn.regression(net, optimizer='adam', learning_rate=learning_rate,
                         loss='mean_square')
model = tflearn.DNN(net, tensorboard_verbose=3)

model.fit(X_train, y_train, batch_size=batch_size, n_epoch=training_iters)

IndexError: list index out of range

In [ ]:
model.fit(X_test, y_test, batch_size=batch_size, n_epoch=training_iters)